In [1]:
import numpy as np
import sounddevice as sd
from scipy import signal as sig

In [2]:
sd.query_devices()
sd.default.device = 'BlackHole 16ch, Core Audio' #only for scree-rec

In [3]:
# Instruments

sr = 44100
duration = 160 

def kick(frq, dur):
    line = np.linspace(0, 1, int((sr / 1000) * dur))
    line2 = np.sqrt(line)
    line3 = line2 * frq - 0.15
    line4 = np.cos(line3)
    envexp = 0.5 ** (25 * line)
    kick = line4 * envexp
    sos = sig.butter(2, 300, 'lp', analog=False, fs=1000, output='sos')
    filtered = sig.sosfilt(sos, kick)
    return filtered * 3

KICK = kick(30, duration)


def snare(frq, dur):
    noise = np.random.random_sample(int((sr / 1000) * dur)) * 2 - 1
    line = np.linspace(0, 1, int((sr / 1000) * dur))
    envexp = 0.5 ** (12.5 * line)
    sos = sig.butter(4, 20, 'hp', analog=False, fs=1000, output='sos')
    filtered = sig.sosfilt(sos, (noise * envexp))
    sos = sig.butter(1, [5, 40], 'bp', fs=1000, output='sos')
    filtered_2 = sig.sosfilt(sos, filtered)
    def sine_tone(frq, dur):
        sr = 44100
        line = np.linspace(0, 1, int((sr / 1000) * dur))
        t = np.arange(int((sr / 1000) * dur)) / sr
        envexp = 0.5 ** (25 * line)
        sine = 1 * np.sin(2 * np.pi * frq * t) * envexp
        return sine
    snare = (filtered_2 + sine_tone(frq, dur)) * 4
    return snare

SNARE = snare(250, duration)


def hi_hat(dur):
    line = np.linspace(1, 0, int((sr / 1000) * dur))
    line2 = line ** 4
    def square_tone(frq, dur):
        sr = 44100
        line = np.linspace(0, 1, int((sr / 1000) * dur))
        t = np.arange(int((sr / 1000) * dur)) / sr
        envexp = 0.5 ** (25 * line)
        sine = 1 * np.sin(2 * np.pi * frq * t)
        square = np.where(sine > 0, 1, -1) * envexp
        return square
    noise = np.random.random_sample(int((sr / 1000) * dur)) * 2 - 1
    high_noise = square_tone(350, dur) + square_tone(800, dur) + (noise / 4)
    sos = sig.butter(10, 100, 'hp', analog=False, fs=1000, output='sos')
    filtered = sig.sosfilt(sos, high_noise)
    sos = sig.butter(2, 100, 'hp', analog=False, fs=1000, output='sos')
    filtered_2 = sig.sosfilt(sos, filtered)
    line3 = filtered_2 * line2 * 4
    return line3

HIHAT = hi_hat(duration)


def open_hat(dur):
    line = np.linspace(1, 0, int((sr / 1000) * dur))
    line2 = line ** 0.1
    def square_tone(frq, dur):
        sr = 44100
        line = np.linspace(0, 1, int((sr / 1000) * dur))
        t = np.arange(int((sr / 1000) * dur)) / sr
        envexp = 0.5 ** (25 * line)
        sine = 1 * np.sin(2 * np.pi * frq * t)
        square = np.where(sine > 0, 1, -1) * envexp
        return square
    noise = np.random.random_sample(int((sr / 1000) * dur)) * 2 - 1
    high_noise = square_tone(350, dur) + square_tone(800, dur) + (noise / 4)
    sos = sig.butter(10, 50, 'hp', analog=False, fs=1000, output='sos')
    filtered = sig.sosfilt(sos, high_noise)
    sos = sig.butter(2, 50, 'hp', analog=False, fs=1000, output='sos')
    filtered_2 = sig.sosfilt(sos, filtered)
    line3 = filtered_2 * line2 * 4
    return line3
    
OPENHAT = open_hat(duration)


def wood_block(frq, ratio, amount, dur):
    def sine_tone(frq, dur):
        sr = 44100
        line = np.linspace(0, 1, int((sr / 1000) * dur))
        t = np.arange(int((sr / 1000) * dur)) / sr
        envexp = 0.5 ** (25 * line)
        sine = 1 * np.sin(2 * np.pi * frq * t) * envexp
        return sine
    fm = frq + sine_tone(frq * ratio, dur) * amount
    sr = 44100
    line = np.linspace(0, 1, int((sr / 1000) * dur))
    t = np.arange(int((sr / 1000) * dur)) / sr
    envexp = 0.5 ** (25 * line)
    sine = 1 * np.sin(2 * np.pi * fm * t) * envexp
    return sine
    
WOODBLOCK = wood_block(880, 2.25, 80, duration)


def mid_tom(frq, dur):
    line = np.linspace(1, 0, int((sr / 1000) * dur))
    line2 = line ** 2.5
    freq = np.linspace(np.sqrt(frq + (frq * 0.5)), np.sqrt(frq), int((sr / 1000) * dur))
    freq2 = freq ** 2
    t = np.arange(int((sr / 1000) * dur)) / sr
    sine = 1 * np.sin(2 * np.pi * freq2 * t) * line2 * 2.5
    noise = np.random.random_sample(int((sr / 1000) * dur)) * 2 - 1
    sos = sig.butter(10, 70, 'hp', analog=False, fs=1000, output='sos')
    filtered = sig.sosfilt(sos, noise)
    sos = sig.butter(2, 30, 'lp', analog=False, fs=1000, output='sos')
    filtered_2 = sig.sosfilt(sos, filtered)
    tom = sine + ((filtered_2 * line2) * 0.085)
    return tom

MIDTOM = mid_tom(175, duration)


# simple panning - algorithm
def panner(x, angle):
    # pan a mono audio source into stereo
    # x is a numpy array, angle is the angle in radiants
    left = np.sqrt(2)/2.0 * (np.cos(angle) - np.sin(angle)) * x
    right = np.sqrt(2)/2.0 * (np.cos(angle) + np.sin(angle)) * x
    return np.dstack((left,right))[0]


def pause(note):
    pause = np.zeros_like(note)
    return pause

In [4]:
# Sequencer

kick_pat = '*------**-----*-'
snare_pat = '----*-------*--*'
hihat_pat = '**-*-*-***-*-*--'
open_hat_pat = '--*---*---*---*-'
wood_block_pat = '-*---*-*-*---***'
mid_tom_pat = '---*--*----*--*-'

In [5]:
kick_seq = np.concatenate([KICK if char == '*' else pause(KICK) for char in kick_pat])
snare_seq = np.concatenate([SNARE if char == '*' else pause(SNARE) for char in snare_pat])
hihat_seq = np.concatenate([HIHAT if char == '*' else pause(HIHAT) for char in hihat_pat])
open_hat_seq = np.concatenate([OPENHAT if char == '*' else pause(OPENHAT) for char in open_hat_pat])
wood_block_seq = np.concatenate([WOODBLOCK if char == '*' else pause(WOODBLOCK) for char in wood_block_pat])
mid_tom_seq = np.concatenate([MIDTOM if char == '*' else pause(MIDTOM) for char in mid_tom_pat])

In [6]:
rep = 16

panning_values = [0.03, 0, -15, 15, -35, 35]
instrument_seq = [kick_seq, snare_seq, hihat_seq, open_hat_seq, wood_block_seq, mid_tom_seq]
panned_instruments = [panner(instrument_seq[i], panning_values[i]) for i in range(len(panning_values))]
vol_mix_values = [1, 1, 0.4, 0.35, 0.6, 0.6]
vol_pan_inst = [np.tile(panned_instruments[j] * vol_mix_values[j], (rep, 1)) for j in range(len(vol_mix_values))]
beat = sum(vol_pan_inst) 

In [7]:
sd.play(beat, sr)

In [8]:
sd.stop()